# Analyze German WIKIPEDIA Corpus

In [1]:
from rwse_checker.rwse import RWSE_Checker
from transformers import AutoTokenizer
from helper import check_token

import os
import pandas as pd
import spacy

/Users/dgardner/GitHub/catalpa-cl/rwse-experiments/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rwse = RWSE_Checker(language='de')
nlp = spacy.load('de_core_news_sm')

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.

## Cleaning

In [3]:
collection = dict()
with open('input/dataset_semantic_de.txt', 'r') as f:
    result = []
    idx = 0
    for line in f:
        if line == "\n":
            collection[idx] = result
            idx += 1
            result = []
        else:
            result.append(line.strip())

len(collection)

1407

In [4]:
collection[0]

['7',
 '1761615',
 'jetztigen',
 'jetzigen',
 '650',
 'Wirtschaftssystem oder Politik oder Handlungen Medizin und Erziehungssystem . Dies gelingt nur, wenn die einzelnen Einheiten aneinander anschlussfähig sind, was durch einen systemspezifischen Code geleistet wird, der als zentrale Logik Leitunterscheidung aller Komunikation zugrunde liegt und sie als systemzugehörig erkennbar macht. Im Wirtschaftssystem beispielsweise sorgt der Code Zahlen/nicht Zahlen dafür, dass die Kommunikationen sich auf sich selbst beziehen, und sich selbst reproduzieren kann, also dass auf jede Zahlung eine neue erfolgt. Dies funktioniert über das generalisierte Kommunikationsmedium Geld, das die letzte Zahlung mit der jetztigen verknüpft.Würde das Geld nicht mehr aktzeptiert, folgt der Zahlung keine weitere Zahlung mehr und das System hätte seine Anschlussfähigkeit verloren. Die Anschlussfähigkeit innerhalb eines Systems wird als Selbstreferenz bezeichnet, im Gegensatz zum fremdreferentiellen Bezug auf die U

In [5]:
file_name = 'input/cleaned_dataset_semantic_de.csv'

def clean_text(text):
    text = '. '.join([part.strip() for part in text.split('.')]).strip()
    text = '. '.join([part.strip() for part in text.split('?')]).strip()
    return text

def find_sentence(token_text, token_start, text):
    tmp_doc = nlp(clean_text(text))
    for sent in tmp_doc.sents:
        if sent.start_char <= token_start < sent.end_char:
            for tmp_token in sent:
                if tmp_token.text == token_text:
                    tmp_start = tmp_token.idx - sent.start_char
                    tmp_end = tmp_start + len(tmp_token.text)
                    return sent.text[:tmp_start] + '[MASK]' + sent.text[tmp_end:]
    return None

if not os.path.exists(file_name):

    checkpoint = 'bert-base-multilingual-cased'
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    cleaned_collection_semantic = []
    bad_words = []
    bad_indices = []
    for key, value in collection.items():
        item = {
            'org_index': int(value[1]),
            'org_token': value[2],
            'org_suggestion': value[3],
            'confusion_set': f'{",".join(sorted([value[2],value[3]]))}',
            'text': find_sentence(value[2], int(value[4]), ' '.join(value[5:]))
        }
        if check_token(tokenizer, item['org_token']) is None:
            bad_words.append(item['org_token'])
        elif check_token(tokenizer, item['org_suggestion']) is None:
            bad_words.append(item['org_suggestion'])
        elif item['text'] is None:
            bad_indices.append(item['org_index'])
        else:
            cleaned_collection_semantic.append(item)


    cleaned_collection_semantic = pd.DataFrame.from_dict(cleaned_collection_semantic)
    for confusion_set in cleaned_collection_semantic['confusion_set'].unique():
        indices = cleaned_collection_semantic[cleaned_collection_semantic['confusion_set'] == confusion_set].index
        if len(indices) == 2:
            cleaned_collection_semantic.drop(min(indices), inplace=True)
    cleaned_collection_semantic.to_csv(file_name, index=False, sep='\t')
    pd.DataFrame(bad_words).to_csv(file_name+'-bad_words', index=False, header=None, sep='\t')
    pd.DataFrame(bad_indices).to_csv(file_name+'-bad_indices', index=False, header=None, sep='\t')
else:
    cleaned_collection_semantic = pd.read_csv(file_name, sep='\t')

cleaned_collection_semantic

,org_index,org_token,org_suggestion,confusion_set,text
0,41103181,Universität,University,"University,Universität","Möglich ist aber auch, dass es „CIS-co“ war – ..."
1,40668358,Resistance,Résistance,"Resistance,Résistance",Wie viele britische Regisseure leistete Hitchc...
2,15975937,sin,sind,"sin,sind",Nahezu alle Hochfesten Aluminiumlegierungen [M...
3,3929925,vor,fort,"fort,vor",Dareios setzte inzwischen seine Flucht [MASK].
4,3929925,Herr,Heer,"Heer,Herr","Erst als der Monsun wieder begann, erreichte d..."
...,...,...,...,...,...
219,45707570,Zwei,Zweig,"Zwei,Zweig",Popmusik ist der lukrativste [MASK] der Musiki...
220,38794857,Texte,Werke,"Texte,Werke",Nach deutschem Recht wird der Begriff Gemeinfr...
222,30194707,Brad,Brett,"Brad,Brett","Einmal, bevor er [MASK] erschießt und einmal, ..."
223,30875256,Buch,Butch,"Buch,Butch",Als Butch Maynard ersticht und dann Zed mit de...


## Analysis

In [6]:
file_name = 'output/report_dataset_semantic_de.csv'
input_file_name = 'input/modified_dataset_semantic_de.csv'

if not os.path.exists(file_name):
    data = pd.read_csv(input_file_name, sep='\t')
    with (open(file_name, 'w') as f):
        print('result_fw', 'result_bw', 'index' ,'org_token', 'org_suggestion', 'masked_sentence', sep='\t', end='\n', file=f)
        for index, item in data.iterrows():
            org_token = item['org_token']
            org_suggestion = item['org_suggestion']
            sentence = item['text']

            rwse.set_confusion_sets([item['confusion_set'].split(',')])
            suggestion, certainty = rwse.check(org_token, sentence)
            # Used in miss rate analysis
            result_fw = 'TP' if suggestion == org_suggestion else 'FN'

            # Switch tokens
            org_token, org_suggestion = org_suggestion, org_token
            suggestion, certainty = rwse.check(org_token, sentence)
            # Used in false-alarm rate analysis
            result_bw = 'FP' if suggestion == org_suggestion else 'TN'

            print(result_fw, result_bw, item['org_index'], item['org_token'], item['org_suggestion'], sentence, sep='\t', end='\n', file=f)

classification_results = pd.read_csv(file_name, sep='\t')
classification_results

,result_fw,result_bw,index,org_token,org_suggestion,masked_sentence
0,TP,TN,3929925,Herr,Heer,"Erst als der Monsun wieder begann, erreichte d..."
1,TP,TN,21426165,After,Alter,Im [MASK] von sechs Jahren wurde Turing nach S...
2,TP,TN,26295245,lies,ließ,Auch gegenüber den zahlreichen vegetarischen G...
3,TP,TN,47872734,Wegen,Wagen,Im Römischen Reich wurden um 200 n. Wagen benu...
4,TP,TN,14229212,erhalten,enthalten,Ebenso beliebt sind die kartoffeln Sieglinde a...
5,TP,TN,30929844,erhalten,gehalten,Sie wurden zunächst für einen Zweig des Kuschi...
6,TP,TN,51767464,westlich,wesentlich,Viele HFA-Autisten sind deshalb als Erwachsene...
7,TP,TN,60483276,wird,wirkt,Biologische Bedeutung Elementares Chlor [MASK]...
8,TP,TN,46454650,Sommer,Somme,Frankreich setzte als erste der kriegführenden...
9,TP,TN,8306042,Ecke,Erde,"Duergar Die Duergar, auch Dunkelzwerge oder Gr..."


In [7]:
summarized_results = {
    'TP': len(classification_results[classification_results['result_fw'] == 'TP']),
    'FN': len(classification_results[classification_results['result_fw'] == 'FN']),
    'TN': len(classification_results[classification_results['result_bw'] == 'TN']),
    'FP': len(classification_results[classification_results['result_bw'] == 'FP']),
}
print(summarized_results)

false_alarm_rate = summarized_results['FP']/len(classification_results)
print(f'false-alarm rate: {false_alarm_rate:.3f}')

miss_rate = summarized_results['FN']/len(classification_results)
print(f'miss rate: {miss_rate:.3f}')

accuracy = (summarized_results['TP']+summarized_results['TN'])/(len(classification_results) * 2)
print(f'accuracy: {accuracy:.3f}')

{'TP': 27, 'FN': 3, 'TN': 30, 'FP': 0}
false-alarm rate: 0.000
miss rate: 0.100
accuracy: 0.950
